# 基准测试

Hugging Face 的基准测试工具已弃用，建议使用外部基准测试库来测量 Transformer 模型的速度和内存复杂度。

让我们看看如何对 🤗 Transformers 模型进行基准测试，最佳实践以及已经可用的基准测试。

详细说明如何对 🤗 Transformers 模型进行基准测试的笔记本可以在这里找到：[链接](https://github.com/huggingface/notebooks/tree/main/examples/benchmark.ipynb)。

## 如何对 🤗 Transformers 模型进行基准测试

类 `PyTorchBenchmark` 和 `TensorFlowBenchmark` 允许灵活地对 🤗 Transformers 模型进行基准测试。这些基准测试类允许我们测量 **峰值内存使用** 和 **所需时间**，包括 **推理** 和 **训练**。

这里，**推理** 定义为一次前向传递，而 **训练** 定义为一次前向传递和一次反向传递。

基准测试类 `PyTorchBenchmark` 和 `TensorFlowBenchmark` 在实例化时分别需要一个 `PyTorchBenchmarkArguments` 或 `TensorFlowBenchmarkArguments` 对象。`PyTorchBenchmarkArguments` 和 `TensorFlowBenchmarkArguments` 是数据类，包含其对应基准测试类的所有相关配置。以下示例展示了如何对类型为 _bert-base-cased_ 的 BERT 模型进行基准测试。

### PyTorch


In [ ]:
from transformers import PyTorchBenchmark, PyTorchBenchmarkArguments

args = PyTorchBenchmarkArguments(
    models=["google-bert/bert-base-uncased"],
    batch_sizes=[8],
    sequence_lengths=[8, 32, 128, 512]
)
benchmark = PyTorchBenchmark(args)


### TensorFlow


In [ ]:
from transformers import TensorFlowBenchmark, TensorFlowBenchmarkArguments

args = TensorFlowBenchmarkArguments(
    models=["google-bert/bert-base-uncased"],
    batch_sizes=[8],
    sequence_lengths=[8, 32, 128, 512]
)
benchmark = TensorFlowBenchmark(args)


这里，基准测试参数数据类有三个参数：`models`、`batch_sizes` 和 `sequence_lengths`。参数 `models` 是必需的，期望一个包含来自 [模型中心](https://huggingface.co/models) 的模型标识符的列表。参数 `batch_sizes` 和 `sequence_lengths` 定义了模型基准测试中 `input_ids` 的大小。通过基准测试参数数据类可以配置许多其他参数。有关这些参数的详细信息，可以直接查阅文件 `src/transformers/benchmark/benchmark_args_utils.py`、`src/transformers/benchmark/benchmark_args.py`（对于 PyTorch）和 `src/transformers/benchmark/benchmark_args_tf.py`（对于 TensorFlow）。或者，从根目录运行以下 shell 命令将分别打印出 PyTorch 和 TensorFlow 的所有可配置参数的描述列表。

### PyTorch


In [ ]:
python examples/pytorch/benchmarking/run_benchmark.py --help


实例化的基准测试对象可以通过调用 `benchmark.run()` 简单地运行。


In [ ]:
results = benchmark.run()
print(results)


### TensorFlow


In [ ]:
python examples/tensorflow/benchmarking/run_benchmark_tf.py --help


实例化的基准测试对象可以通过调用 `benchmark.run()` 简单地运行。


In [ ]:
results = benchmark.run()
print(results)


默认情况下，会基准测试 **推理** 所需的 **时间** 和 **内存**。在上面的示例输出中，前两部分显示了 **推理时间** 和 **推理内存** 的结果。此外，第三部分 **环境信息** 列出了与计算环境相关的所有相关信息，例如 GPU 类型、系统、库版本等。这些信息可以选保存在一个 `.csv` 文件中，只需在 `PyTorchBenchmarkArguments` 和 `TensorFlowBenchmarkArguments` 中添加参数 `save_to_csv=True`。在这种情况下，每个部分将保存在一个单独的 `.csv` 文件中。可以通过参数数据类定义每个 `.csv` 文件的路径。

除了通过模型标识符（例如 `google-bert/bert-base-uncased`）基准测试预训练模型外，用户还可以基准测试任何可用模型类的任意配置。在这种情况下，必须插入一个配置列表，如下所示。

### PyTorch


In [ ]:
from transformers import PyTorchBenchmark, PyTorchBenchmarkArguments, BertConfig

args = PyTorchBenchmarkArguments(
    models=["bert-base", "bert-384-hid", "bert-6-lay"],
    batch_sizes=[8],
    sequence_lengths=[8, 32, 128, 512]
)
config_base = BertConfig()
config_384_hid = BertConfig(hidden_size=384)
config_6_lay = BertConfig(num_hidden_layers=6)

benchmark = PyTorchBenchmark(args, configs=[config_base, config_384_hid, config_6_lay])
benchmark.run()


### TensorFlow


In [ ]:
from transformers import TensorFlowBenchmark, TensorFlowBenchmarkArguments, BertConfig

args = TensorFlowBenchmarkArguments(
    models=["bert-base", "bert-384-hid", "bert-6-lay"],
    batch_sizes=[8],
    sequence_lengths=[8, 32, 128, 512]
)
config_base = BertConfig()
config_384_hid = BertConfig(hidden_size=384)
config_6_lay = BertConfig(num_hidden_layers=6)

benchmark = TensorFlowBenchmark(args, configs=[config_base, config_384_hid, config_6_lay])
benchmark.run()


同样，默认情况下会测量 **推理** 所需的 **时间** 和 **内存**，但这次是针对 `BertModel` 类的自定义配置。此功能在决定模型应以哪种配置进行训练时特别有用。

## 基准测试最佳实践

本节列出了在基准测试模型时应了解的一些最佳实践。

- 目前仅支持单设备基准测试。当在 GPU 上进行基准测试时，建议用户通过设置 `CUDA_VISIBLE_DEVICES` 环境变量指定代码应在哪个设备上运行，例如在运行代码之前执行 `export CUDA_VISIBLE_DEVICES=0`。
- 选项 `no_multi_processing` 只应在测试和调试时设置为 `True`。为了确保准确的内存测量，建议在单独的进程中运行每个内存基准测试，即确保 `no_multi_processing` 设置为 `True`。
- 分享模型基准测试结果时，应始终说明环境信息。不同的 GPU 设备、库版本等会导致结果有很大差异，因此单独的基准测试结果对社区来说不太有用。

## 分享你的基准测试

之前所有的核心模型（当时有 10 个）都进行了 **推理时间** 的基准测试，涵盖了多种不同的设置：使用 PyTorch、有无 TorchScript、使用 TensorFlow、有无 XLA。所有这些测试都在 CPU（除 TensorFlow XLA 外）和 GPU 上进行了。

该方法在 [这篇博客文章](https://medium.com/huggingface/benchmarking-transformers-pytorch-and-tensorflow-e2917fb891c2) 中有详细介绍，结果可以在 [这里](https://docs.google.com/spreadsheets/d/1sryqufw2D0XlUH4sq3e9Wnxu5EAQkaohzrJbd5HdQ_w/edit?usp=sharing) 查看。

使用新的 **基准** 工具，分享你的基准测试结果比以往任何时候都更容易。

- [PyTorch 基准测试结果](https://github.com/huggingface/transformers/tree/main/examples/pytorch/benchmarking/README.md)
- [TensorFlow 基准测试结果](https://github.com/huggingface/transformers/tree/main/examples/tensorflow/benchmarking/README.md)